In [1]:
pip install pandas

  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   -- ------------------------------------- 0.8/11.6 MB 3.3 MB/s eta 0:00:04
   -------- ------------------------------- 2.4/11.6 MB 5.4 MB/s eta 0:00:02
   --------------------------- ------------ 8.1/11.6 MB 13.6 MB/s eta 0:00:01
   ---------------------------------------- 11.6/11.6 MB 16.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   ------- -------------------------------- 2.4/12.9 MB 13.4 MB/s eta 0:00:01
   --------- ------------------------------ 3.1/12.9 MB 10.3 MB/s eta 0:00:01
   --------- ------------------------------ 3.1/12.9 MB 10.3 MB/s eta 0:00:01
   --------- ------------------------------ 3.1/12.9 MB 10.3 MB/s eta 0:00:01
   --------- ------------------------------ 3.1/12.9 MB 10.3 MB/s eta 0:00:01
   ---------- -------

In [2]:
pip install pymongo

   ---------------------------------------- 0.0/582.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/582.0 kB ? eta -:--:--
   ---------------------------------------- 582.0/582.0 kB 2.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import requests
from pymongo import MongoClient
from datetime import datetime


In [6]:
# URL des données
url_1 = 'https://airqino-api.magentalab.it/v3/getStationHourlyAvg/283181971'  # Remplacez par le lien réel
url_2 = 'https://airqino-api.magentalab.it/v3/getStationHourlyAvg/283164601'  # Remplacez par le lien réel


In [7]:
# Récupérer les données depuis le lien
response_1 = requests.get(url_1)
data_1 = response.json()
response_2 = requests.get(url_2)
data_2 = response.json()

In [8]:
# Convertir la liste de données en DataFrame
df_1 = pd.DataFrame(data_1["data"])
df_2 = pd.DataFrame(data_2["data"])

In [9]:
df_1['timestamp'] = pd.to_datetime(df_1['timestamp'])
df_2['timestamp'] = pd.to_datetime(df_2['timestamp'])

In [12]:
df_1['date_1'] = df_1['timestamp'].dt.date
df_2['date_2'] = df_2['timestamp'].dt.date

In [15]:
daily_means_1 = df_1.groupby('date_1').agg({'CO': 'mean', 'PM2.5': 'mean'}).reset_index()
daily_means_2 = df_2.groupby('date_2').agg({'CO': 'mean', 'PM2.5': 'mean'}).reset_index()

In [21]:
daily_means_1['date_1'] = pd.to_datetime(daily_means_1['date_1'])
daily_means_2['date_2'] = pd.to_datetime(daily_means_2['date_2'])


In [22]:
# Afficher les résultats
print(daily_means_1)
print(daily_means_2)

        date_1        CO      PM2.5
0   2023-09-10  0.031795  12.780000
1   2023-09-11  0.034917  11.078584
2   2023-09-12  0.227283  12.922088
3   2023-09-13  0.209933  14.277363
4   2023-09-14  0.150275  14.533692
..         ...       ...        ...
343 2024-09-05  0.139701  13.180421
344 2024-09-06  0.118769  14.648611
345 2024-09-07  0.107480  14.217473
346 2024-09-08  0.128420  12.615905
347 2024-09-09  0.198864  15.024578

[348 rows x 3 columns]
        date_2        CO      PM2.5
0   2023-09-10  0.031795  12.780000
1   2023-09-11  0.034917  11.078584
2   2023-09-12  0.227283  12.922088
3   2023-09-13  0.209933  14.277363
4   2023-09-14  0.150275  14.533692
..         ...       ...        ...
343 2024-09-05  0.139701  13.180421
344 2024-09-06  0.118769  14.648611
345 2024-09-07  0.107480  14.217473
346 2024-09-08  0.128420  12.615905
347 2024-09-09  0.198864  15.024578

[348 rows x 3 columns]


In [26]:
# Connexion à MongoDB
client = MongoClient('mongodb://localhost:27017/')  # Remplacez par l'URI de votre MongoDB
db_1 = client['air_quality_db']
collection_1 = db_1['daily_averages_1']
db_2 = client['air_quality_db']
collection_2 = db_2['daily_averages_2']

In [29]:
# Convertir le DataFrame en dictionnaire et insérer les données dans MongoDB
records_1 = daily_means_1.to_dict('records')
collection_1.insert_many(records_1)

InsertManyResult([ObjectId('66def9bcee730d8dd740e452'), ObjectId('66def9bcee730d8dd740e453'), ObjectId('66def9bcee730d8dd740e454'), ObjectId('66def9bcee730d8dd740e455'), ObjectId('66def9bcee730d8dd740e456'), ObjectId('66def9bcee730d8dd740e457'), ObjectId('66def9bcee730d8dd740e458'), ObjectId('66def9bcee730d8dd740e459'), ObjectId('66def9bcee730d8dd740e45a'), ObjectId('66def9bcee730d8dd740e45b'), ObjectId('66def9bcee730d8dd740e45c'), ObjectId('66def9bcee730d8dd740e45d'), ObjectId('66def9bcee730d8dd740e45e'), ObjectId('66def9bcee730d8dd740e45f'), ObjectId('66def9bcee730d8dd740e460'), ObjectId('66def9bcee730d8dd740e461'), ObjectId('66def9bcee730d8dd740e462'), ObjectId('66def9bcee730d8dd740e463'), ObjectId('66def9bcee730d8dd740e464'), ObjectId('66def9bcee730d8dd740e465'), ObjectId('66def9bcee730d8dd740e466'), ObjectId('66def9bcee730d8dd740e467'), ObjectId('66def9bcee730d8dd740e468'), ObjectId('66def9bcee730d8dd740e469'), ObjectId('66def9bcee730d8dd740e46a'), ObjectId('66def9bcee730d8dd740e4

In [28]:
# Convertir le DataFrame en dictionnaire et insérer les données dans MongoDB
records_2 = daily_means_2.to_dict('records')
collection_2.insert_many(records_2)

InsertManyResult([ObjectId('66def9b7ee730d8dd740e2f6'), ObjectId('66def9b7ee730d8dd740e2f7'), ObjectId('66def9b7ee730d8dd740e2f8'), ObjectId('66def9b7ee730d8dd740e2f9'), ObjectId('66def9b7ee730d8dd740e2fa'), ObjectId('66def9b7ee730d8dd740e2fb'), ObjectId('66def9b7ee730d8dd740e2fc'), ObjectId('66def9b7ee730d8dd740e2fd'), ObjectId('66def9b7ee730d8dd740e2fe'), ObjectId('66def9b7ee730d8dd740e2ff'), ObjectId('66def9b7ee730d8dd740e300'), ObjectId('66def9b7ee730d8dd740e301'), ObjectId('66def9b7ee730d8dd740e302'), ObjectId('66def9b7ee730d8dd740e303'), ObjectId('66def9b7ee730d8dd740e304'), ObjectId('66def9b7ee730d8dd740e305'), ObjectId('66def9b7ee730d8dd740e306'), ObjectId('66def9b7ee730d8dd740e307'), ObjectId('66def9b7ee730d8dd740e308'), ObjectId('66def9b7ee730d8dd740e309'), ObjectId('66def9b7ee730d8dd740e30a'), ObjectId('66def9b7ee730d8dd740e30b'), ObjectId('66def9b7ee730d8dd740e30c'), ObjectId('66def9b7ee730d8dd740e30d'), ObjectId('66def9b7ee730d8dd740e30e'), ObjectId('66def9b7ee730d8dd740e3